## 1. Importar las Librerías Necesarias

* Crear un Dataframe con los resultados de los modelos y sus hiperparámetros.
* Seleccionar de ese dataframe el que dió mejor resultado.

In [ ]:
import warnings

from statistics import stdev, mean

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold

from lightgbm import LGBMClassifier, plot_metric

from sklearn.metrics import cohen_kappa_score

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

## 2. Leer los Datos
Al menos los datos Tabulares de la base de "train"

### 2.1. Carga de datos

In [ ]:
datos = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
datos.head()

### 2.2. Información sobre los datos

In [ ]:
datos.info()

## 3. Pre-procesar Nulos
Verificar la existencia de Nulos y decidir como Imputarlos en caso de que existan

Verificar la existencia de Ceros u otros valores que puedan indicar que pueden ser perdidos

### 3.1. Verificación de nulos

De la inspección inicial se identificaron valores de tipo de dato "NaN" en dos variables, **"Name"** y **"Description"**.

In [ ]:
datos.isnull().sum()

#### 3.1.1. Variable "Name"

La columna **"Name"** tiene **1257** valores nulos y **9060** valores únicos. Los valores no son siempre nombres, hay casos que corresponden a descripciones, por ejemplo "4 PUPPIES FOR ADOPTION". Por el momento se decide dejarla fuera del análisis dado que requiere una limpieza detallada para obtener información útil.

In [ ]:
datos["Name"].value_counts()

In [ ]:
datos[datos.Name.isnull()].head()

#### 3.1.2. Variable "Description"

Se decide quitar la variable dado que en el modelo inicial no se a incluir procesamiento de lenguaje natural (ver punto 4).

In [ ]:
datos.Description.describe()

In [ ]:
datos.Description.head()

### 3.2. Verificación de valores perdidos

#### 3.2.1. Valores únicos por columna

In [ ]:
datos.nunique()

#### 3.2.2. Inspección visual de valores

#### Variables categóricas y numéricas con pocos valores distintos

De la inspección visual se concluye que los valores de las variables con menos de 40 valores distintos coinciden con la [descripción presente en los metadatos](https://www.kaggle.com/c/petfinder-adoption-prediction/data).

In [ ]:
# Se seleccionan columnas con menos de 40 valores distintos
datos_a_visualizar = datos.nunique()[datos.nunique() < 40].index

# Se configuran los parámetros de figura a crear, se divide por 2.54 para pasar las dimensiones a cm
plt.rcParams['figure.figsize'] = [30/2.54, 300/2.54]
fix, axs = plt.subplots(nrows = len(datos_a_visualizar), ncols = 1)

# Se crean los gráficos para cada variable
for i, column in enumerate(datos_a_visualizar):
    datos[column].value_counts().plot.bar(title = column, rot = 45, ax = axs[i])

## 4. Convertir o eliminar las Columnas Categóricas

Por ejemplo, la Descripción habría que sacarla para un análisis independiente

### 4.1. Se quitan las columnas Name, Description y PetID

In [ ]:
datos.drop(["Name", "Description", "PetID"], axis = 1, inplace = True)
datos.head()

### 4.2. Variable RescuerID

Reemplazamos el ID del/la rescatador/a por la cantidad de mascotas rescatadas por la persona creando la nueva variable **rescuer_score**.

In [ ]:
rescuer_id = dict(datos['RescuerID'].value_counts())

datos = datos.replace(rescuer_id)

datos.rename(columns= {'RescuerID': 'rescuer_score'}, inplace= True)

datos.head()

## 5. Normalizar o Estandarizar las variables Numericas (para los modelos que sean necesarios)

Revisar si existen valores extremos y considerarlos para los modelos que afecte

Elegimos el algoritmo LGBM que no demanda la normalización o estandarización de las variables.

## 6. Separa la base de Test (10%) y Train (90%)
Pueden ser otros porcentajes que les parezcan mejor

In [ ]:
def crear_train_test(datos: pd.DataFrame, var_respuesta: str, tamanio: float, semilla: int=1) -> tuple:
    
    x_train, x_test, y_train, y_test = train_test_split(
        datos.drop(var_respuesta, axis=1),
        datos[[var_respuesta]], 
        test_size=tamanio, 
        random_state=semilla
    )
    
    return x_train, x_test, y_train, y_test

# Se ejecuta la función a modo de ejemplo, se vuelve a emplear mas abajo.
x_train, x_test, y_train, y_test = crear_train_test(datos, "AdoptionSpeed", 0.1)

### 7. Para la parte de Train, armar un esquema de Cross Validation

Usar 10 Folds

### Esquema propuesto

![Esquema 10 fold CV](https://i.imgur.com/3XSPeOU.png)

### Funciones de ayuda




In [ ]:
def crear_folds(datos: pd.DataFrame, nro_folds: int) -> list:

    kf = KFold(n_splits=nro_folds, shuffle = True,random_state=512)
    folds = [(datos.iloc[train_idx].index, datos.iloc[valid_idx].index) for train_idx, valid_idx in kf.split(datos)]
    
    return folds


def calcular_kappa(y_true, y_pred) -> tuple:
    
    res = cohen_kappa_score(y_true, y_pred.reshape((y_true.shape[0], 5), order="F").argmax(axis=1), weights= 'quadratic')
    
    return "kappa", res, True


def calcular_estadisticos(datos: list) -> dict:
       
    media = mean(datos)
    desv_est = stdev(datos)
    rstdev = desv_est/media*100
    
    return {"media": media, "stdev": desv_est, "rstdev": rstdev}


def calcular_factor_everfitting(modelo, nombre_metrica="kappa"):

    factor_overfitting = modelo.evals_result_['training'][nombre_metrica][-1] / modelo.evals_result_['valid_1'][nombre_metrica][-1]

    return factor_overfitting


def realizar_cv_LGBM_kappa(datos: tuple, nro_folds: int, parametros: dict) -> dict:
  
    x_train_val, y_train_val = datos
    
    kappa_folds = []
    valores_pred = pd.Series()
    valores_verd = pd.Series()
    modelos_folds = []
    
    for i, (fold_train, fold_val) in enumerate(crear_folds(x_train_val.join(y_train_val), nro_folds)):
        print(f"--------- Fold {i+1} ---------")
        xt, xv = x_train_val.loc[fold_train], x_train_val.loc[fold_val]
        yt, yv = y_train_val.loc[fold_train], y_train_val.loc[fold_val]
        
        modelo = LGBMClassifier(**parametros, n_estimators=1000, metric="custom")
        modelo.fit(
            xt, yt, 
            eval_set=[(xt, yt), (xv, yv)],
            early_stopping_rounds=50, 
            eval_metric=calcular_kappa,
            verbose=100
        )
        prediccion = pd.Series(modelo.predict(xv), index=xv.index)
        valores_pred = valores_pred.append(prediccion)
        valores_verd = valores_verd.append(yv[yv.columns[0]])
        
        kappa_folds.append(cohen_kappa_score(yv, prediccion, weights= 'quadratic'))
        
        modelos_folds.append(modelo)
    
    kappa_final_cv = cohen_kappa_score(valores_verd, valores_pred, weights= 'quadratic')
    
    # Entrena el modelo por última vez con todos los datos
    modelo_final = LGBMClassifier(**parametros, n_estimators=1000, metric="custom")
    modelo_final.fit(x_train_val, y_train_val, eval_metric=calcular_kappa)
    
    return {"parametros": parametros, 
            "modelo": modelo_final,
            "kappa": kappa_final_cv,
            "folds": nro_folds, 
            "estadisticos_kappa_cv": calcular_estadisticos(kappa_folds),
            "modelos_folds": modelos_folds,
            "factor_overfitting_mean": mean([calcular_factor_everfitting(datos_modelo) for datos_modelo in modelos_folds]),
            "factor_overfitting_stev": stdev([calcular_factor_everfitting(datos_modelo) for datos_modelo in modelos_folds])
           }

### Ejemplo de corrida de CV para un set de hiperparámetros

No es necesario correr este bloque de código, es solo a modo de ejemplo

In [ ]:
# División de datos en Train y test
x_train, x_test, y_train, y_test = crear_train_test(datos, "AdoptionSpeed", 0.1)

# Selección de hiperparámtros de ejemplo
parametros = {'max_depth': 6, 'num_leaves': 70, 'learning_rate': 0.05}

# Sealización de la validación cruzada con 2 folds para probar
resultado = realizar_cv_LGBM_kappa(datos=(x_train, y_train) , nro_folds= 2, parametros = parametros)

# Visualización de resultado
resultado

# Ejemplo de como se aplicaría el modelo final al conjunto de datos de prueba
# cohen_kappa_score(y_test, resultados["modelo"].predict(x_test), weights= 'quadratic')

## 8. Entrenar al menos un Modelo que prefieran y optimizar al menos un Hiperparámetro

### Funciones de ayuda

In [ ]:
def ejecutar_busqueda_de_hiperparametros_con_cv(datos: tuple, folds: int, grilla_hiperparametros: dict) -> dict:
    
    resultado_grid_search = {}
    resultados_prelim = []
    
    for parametros in grilla_hiperparametros:
        print(f"----- {parametros} -----")
        resultado = realizar_cv_LGBM_kappa(datos= datos , nro_folds= folds, parametros= parametros)
        resultados_prelim.append(resultado)
        
    # Obtención de mejor modelo criterio kappa
    mejor_kappa = max([resultado["kappa"] for resultado in resultados_prelim])

    modelo_mejor_kappa = [resultado for resultado in resultados_prelim if resultado["kappa"] == mejor_kappa][0]
    
    # Obtención de mejor modelo criterio menor overfitting
    menor_overfitting = min([resultado["factor_overfitting_mean"] for resultado in resultados_prelim])

    modelo_menor_overfitting = [resultado for resultado in resultados_prelim if resultado["factor_overfitting_mean"] == menor_overfitting][0]

    
    # Mostrar resultados como dataframe
    resultado_data_frame = pd.DataFrame(resultados_prelim)

    resultado_data_frame = resultado_data_frame.join(
        resultado_data_frame["parametros"].apply(pd.Series) # Transforma a los diccionarios en columnas
    ).drop("parametros", axis = 1) # Quita la columna que estaba como diccionario

    resultado_data_frame = resultado_data_frame.join( # Se repiten los pasos para la otra columna con diccionario
        resultado_data_frame["estadisticos_kappa_cv"].apply(pd.Series)
    ).drop("estadisticos_kappa_cv", axis = 1)

    
    # Armado de presentación de resultado de grid search
    resultado_grid_search["modelo_mejor_kappa"] = modelo_mejor_kappa
    resultado_grid_search["modelo_menor_overfitting"] = modelo_menor_overfitting
    resultado_grid_search["resultados"] = resultados_prelim
    resultado_grid_search["resultados_dataframe"] = resultado_data_frame
    
    return resultado_grid_search

### Definición de hiperparámetros a optimizar

In [ ]:
parametros = []
depths = range(2, 50, 5)
num_leaves = range(5, 100, 15)
learning_rate = 0.05

for leaf in num_leaves:
    for depth in depths:
        parametros.append({"max_depth": depth, "num_leaves": leaf, "learning_rate": learning_rate })

# Selección de parámetros aleatorios para random search
        
from random import randint

numeros_aleatorios = list(set([randint(0, len(parametros)-1) for _ in range(0, 10)]))
numeros_aleatorios

parametros = [parametros[i] for i in numeros_aleatorios]
parametros

### Ejecutar random search y CV

In [ ]:
x_train, x_test, y_train, y_test = crear_train_test(datos, "AdoptionSpeed", 0.1)

resultados_random_search_cv = ejecutar_busqueda_de_hiperparametros_con_cv(datos = (x_train, y_train), 
                                                                          folds = 10, 
                                                                          grilla_hiperparametros = parametros)

In [ ]:
resultados_random_search_cv["resultados_dataframe"].sort_values("factor_overfitting_mean")

## Entrenamiento y testeo del modelo final

In [ ]:
## Hiperparámetros seleccionados

parametros_seleccionados = resultados_random_search_cv["modelo_menor_overfitting"]["parametros"]


modelo = LGBMClassifier(**parametros_seleccionados, n_estimators=1000, metric="custom")
modelo.fit(
    x_train, y_train, 
    eval_set=[(x_train, y_train), (x_test, y_test)],
    early_stopping_rounds=50, 
    eval_metric=calcular_kappa,
    verbose=100
)

plt.rcParams['figure.figsize'] = [20/2.54, 20/2.54]
plot_metric(modelo)

In [ ]:
cohen_kappa_score(
    y_test,
    modelo.predict(x_test),
    weights= 'quadratic'
)

## Entrenamiento Final del modelo con el 100 % de los datos de entrenamiento y mejores hiperparámetros

In [ ]:
parametros_seleccionados = resultados_random_search_cv["modelo_menor_overfitting"]["parametros"]

modelo_final = LGBMClassifier(**parametros_seleccionados, n_estimators=1000, metric="custom")
modelo_final.fit(datos.drop("AdoptionSpeed", axis=1), datos[["AdoptionSpeed"]], verbose=100)

## Preparación de submission

In [ ]:
#carga dataset de validacion y transformacion
datos_test_entrega_completo = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')
datos_test_entrega = datos_test_entrega_completo.drop(["Name", "Description", "PetID"], axis = 1)
datos_test_entrega_rescuer_id = dict(datos_test_entrega['RescuerID'].value_counts())
datos_test_entrega = datos_test_entrega.replace(datos_test_entrega_rescuer_id)
datos_test_entrega.rename(columns= {'RescuerID': 'rescuer_score'}, inplace= True)

#prediccion con dataset de validacion
predicciones = modelo_final.predict(datos_test_entrega)
predicciones
#submit de las predicciones
submit = pd.DataFrame.from_dict({'PetID': datos_test_entrega_completo['PetID'],
                                     'AdoptionSpeed': predicciones})
submit.head()
submit.to_csv('submission.csv', index=False)
#chequeo del submit
!head submission.csv
